In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/student_resource/Documentation_template.md
/kaggle/input/dataset/student_resource/README.md
/kaggle/input/dataset/student_resource/sample_code.py
/kaggle/input/dataset/student_resource/dataset/sample_test.csv
/kaggle/input/dataset/student_resource/dataset/sample_test_out.csv
/kaggle/input/dataset/student_resource/dataset/train.csv
/kaggle/input/dataset/student_resource/dataset/test.csv
/kaggle/input/dataset/student_resource/src/utils.py
/kaggle/input/dataset/student_resource/src/example.ipynb


In [2]:
df_test=pd.read_csv('/kaggle/input/dataset/student_resource/dataset/test.csv')

In [3]:
all_links = df_test['image_link']
unique_links = all_links.dropna().unique().tolist()
len(unique_links)

72222

In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sample_id        75000 non-null  int64 
 1   catalog_content  75000 non-null  object
 2   image_link       75000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [5]:
import os
import urllib.request
import urllib.parse
from pathlib import Path
from multiprocessing import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import time
def download_image(image_link, savefolder, retries=3):
    if isinstance(image_link, str) and image_link.strip():
        try:
            filename = Path(urllib.parse.unquote(Path(image_link).name)).name
        except Exception:
            filename = image_link.split('/')[-1]
        image_save_path = os.path.join(savefolder, filename)
        if os.path.exists(image_save_path):
            return
        for attempt in range(retries):
            try:
                req = urllib.request.Request(
                    image_link,
                    headers={'User-Agent': 'Mozilla/5.0'}
                )
                with urllib.request.urlopen(req) as response, open(image_save_path, 'wb') as out_file:
                    out_file.write(response.read())
                return
            except Exception as ex:
                if attempt < retries - 1:
                    continue
                else:
                    print(f'Warning: Failed to download {image_link} after {retries} attempts.\nError: {ex}')
    else:
        print(f'Warning: Invalid or empty image link provided: {image_link}')

def download_images(image_links, download_folder, num_workers=None):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    if num_workers is None:
        num_workers = max(1, cpu_count() - 1)
    print(f"Starting download with {num_workers} parallel workers...")
    download_func = partial(download_image, savefolder=download_folder)
    with Pool(processes=num_workers) as pool:
        list(tqdm(pool.imap_unordered(download_func, image_links), total=len(image_links), desc="Downloading Images"))


In [6]:
IMAGE_DOWNLOAD_FOLDER = 'product_images'
BATCH_SIZE = 10000 
NUM_WORKERS = 20 

In [7]:
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
for i, batch in enumerate(chunk_list(unique_links, BATCH_SIZE)):
   
    download_images(
        image_links=batch,
        download_folder=IMAGE_DOWNLOAD_FOLDER,
        num_workers=NUM_WORKERS
    )
   
    time.sleep(1)


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...


Error: HTTP Error 404: Not Found


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...


Starting download with 20 parallel workers...
